In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
from sklearn.metrics import roc_curve
from scipy import interpolate

In [2]:
import sys
import logging

nblog = open("nb.log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

Autosaving every 5 seconds


In [3]:
os.chdir(r'/home/manhducnmd/pp_dijet/Results_full')

In [4]:
np.random.seed(42)

In [5]:
tf.random.set_seed(42)

In [6]:
def CWoLA(p_i):
    
    input_1 = keras.layers.Input(shape = (p_i,p_i,1), name = "jet_1")
    y = keras.layers.BatchNormalization()(input_1)
    input_2 = keras.layers.Input(shape = (p_i,p_i,1), name = "jet_2")
    z = keras.layers.BatchNormalization()(input_2)
    
    x = keras.layers.Concatenate(axis = 0)([y, z])

    x = keras.layers.Conv2D(64, (5,5), padding = 'same')(input_1)
    x = keras.layers.MaxPool2D(pool_size = (2,2))(x)
        
    x = keras.layers.Conv2D(64, (5, 5), padding='same')(x)
    x = keras.layers.MaxPool2D(pool_size = (2,2))(x)

    x = keras.layers.Conv2D(128, (3,3), padding='same')(x)
    x = keras.layers.MaxPool2D(pool_size = (2,2))(x)
    
    x = keras.layers.Conv2D(128, (3,3))(x)
    x = keras.layers.Flatten()(x)
    
    x = keras.layers.Dense(128, activation = 'relu')(x)
    x = keras.layers.Dense(128, activation = 'relu')(x)
    x = keras.layers.Dense(128, activation = 'relu')(x)
    
    x = keras.layers.Dense(1, activation = 'sigmoid')(x)
    
    
    return keras.Model(         
    inputs=[input_1, input_2],
    outputs=x,)

In [ ]:
n_train_sr_bkg = 25000

mean_results = []
std_results = []
for p_1 in [25]:
    bkg_sr_jet_1 = np.load(f'background_images_sr_{p_1}_jet_1.npy')
    bkg_sr_jet_2 = np.load(f'background_images_sr_{p_1}_jet_2.npy')
    
    bkg_sb_jet_1 = np.load(f'background_images_sb_{p_1}_jet_1.npy')
    bkg_sb_jet_2 = np.load(f'background_images_sb_{p_1}_jet_2.npy')
    
    sb_sr_bkg = np.shape(bkg_sb_jet_1)[0]/np.shape(bkg_sr_jet_2)[0]
    #n_val_sr_bkg = 5000
    n_train_sb_bkg = int(np.round(n_train_sr_bkg*sb_sr_bkg))
    #n_val_sb_bkg = int(np.round(n_val_sr_bkg*sb_sr_bkg))
    
    #print(n_train_sb_bkg, n_val_sb_bkg)
    n_test_sr_bkg = 20000
    #First jet, background, SR
    bkg_train_sr_1 = bkg_sr_jet_1[0:n_train_sr_bkg]
    #bkg_val_sr_1 = bkg_sr_jet_1[n_train_sr_bkg:n_train_sr_bkg+n_val_sr_bkg]
    bkg_test_1 = bkg_sr_jet_1[n_train_sr_bkg:n_train_sr_bkg+n_test_sr_bkg]
    #Second jet, background, SR
    bkg_train_sr_2 = bkg_sr_jet_2[0:n_train_sr_bkg]
    #bkg_val_sr_2 = bkg_sr_jet_2[n_train_sr_bkg:n_train_sr_bkg+n_val_sr_bkg]
    bkg_test_2 = bkg_sr_jet_2[n_train_sr_bkg:n_train_sr_bkg+n_test_sr_bkg]
    #First jet, background, SB
    bkg_train_sb_1 = bkg_sb_jet_1[0:n_train_sb_bkg]
    #bkg_val_sb_1 = bkg_sb_jet_1[n_train_sb_bkg:n_train_sb_bkg+n_val_sb_bkg]
    #Second jet, background, SB
    bkg_train_sb_2 = bkg_sb_jet_2[0:n_train_sb_bkg]
    #bkg_val_sb_2 = bkg_sb_jet_2[n_train_sb_bkg:n_train_sb_bkg+n_val_sb_bkg]

    #print(np.shape(bkg_train_sr_1),np.shape(bkg_val_sr_1),np.shape(bkg_test_1), np.shape(bkg_train_sb_1), np.shape(bkg_val_sb_1))
    #print(np.shape(bkg_train_sr_2),np.shape(bkg_val_sr_2),np.shape(bkg_test_2), np.shape(bkg_train_sb_2), np.shape(bkg_val_sb_2))
    
    #masked_data = np.ma.masked_equal(np.mean(bkg_test_1, axis = 0), 0.)

    # Create a colormap based on 'jet'
    #jet = plt.cm.jet

    # Create a custom colormap with white for masked values
    #colors = jet(np.linspace(0, 1, 256))
    #custom_cmap = mcolors.ListedColormap(colors)
    # Use a logarithmic norm, avoiding zero values
    #norm = mcolors.LogNorm(vmin=10**-3, vmax=10**3)

    # Plot using imshow with the custom colormap and logarithmic normalization
    #plt.figure(figsize = (5,5))
    #plt.imshow(masked_data, cmap=custom_cmap, norm=norm, interpolation='nearest')
    #cbar = plt.colorbar()
    #cbar.set_label("P_T")
    #plt.xlabel("Eta")
    #plt.ylabel("Phi")
    #plt.gca().invert_yaxis()
    #plt.show()
    
    signal_sr_jet_1 = np.load(f'id10_sr_{p_1}_jet_1.npy')
    signal_sr_jet_2 = np.load(f'id10_sr_{p_1}_jet_2.npy')
    
    signal_sb_jet_1 = np.load(f'id10_sb_{p_1}_jet_1.npy')
    signal_sb_jet_2 = np.load(f'id10_sb_{p_1}_jet_2.npy')
    
    #print(np.shape(signal_sr_jet_1), np.shape(signal_sr_jet_2), np.shape(signal_sb_jet_1), np.shape(signal_sb_jet_2))
    
    sb_sr_signal = np.shape(signal_sb_jet_1)[0]/np.shape(signal_sr_jet_1)[0]
    for n_train_sr_signal in [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 1100, 1200]:
        #n_val_sr_signal = int(np.round(n_train_sr_signal/4))

        n_train_sb_signal = int(np.round(n_train_sr_signal*sb_sr_signal))
        #n_val_sb_signal = int(np.round(n_val_sr_signal*sb_sr_signal))
        #print(n_train_sr_signal, n_val_sr_signal)
        n_test_sr_signal = 20000

        #First jet, signal, SR
        signal_train_sr_1 = signal_sr_jet_1[0:n_train_sr_signal]
        #signal_val_sr_1 = signal_sr_jet_1[n_train_sr_signal:n_train_sr_signal+n_val_sr_signal]
        signal_test_1 = signal_sr_jet_1[n_train_sr_signal:n_train_sr_signal+n_test_sr_signal]

        #Second jet, signal, SR
        signal_train_sr_2 = signal_sr_jet_2[0:n_train_sr_signal]
        #signal_val_sr_2 = signal_sr_jet_2[n_train_sr_signal:n_train_sr_signal+n_val_sr_signal]
        signal_test_2 = signal_sr_jet_2[n_train_sr_signal:n_train_sr_signal+n_test_sr_signal]

        #First jet, signal, SB
        signal_train_sb_1 = signal_sb_jet_1[0:n_train_sb_signal]
        #signal_val_sb_1 = signal_sb_jet_1[n_train_sb_signal:n_train_sb_signal+n_val_sb_signal]

        #Second jet, signal, SB
        signal_train_sb_2 = signal_sb_jet_2[0:n_train_sb_signal]
        #signal_val_sb_2 = signal_sb_jet_2[n_train_sb_signal:n_train_sb_signal+n_val_sb_signal]

        #print(np.shape(signal_train_sr_1),np.shape(signal_val_sr_1),np.shape(signal_test_1), np.shape(signal_train_sb_1), np.shape(signal_val_sb_1))
        #print(np.shape(signal_train_sr_2),np.shape(signal_val_sr_2),np.shape(signal_test_2), np.shape(signal_train_sb_2), np.shape(signal_val_sb_2)) 
        
        #masked_data = np.ma.masked_equal(np.mean(signal_train_sr_1, axis = 0), 0.)

        # Create a colormap based on 'jet'
        #jet = plt.cm.jet

        # Create a custom colormap with white for masked values
        #colors = jet(np.linspace(0, 1, 256))
        #custom_cmap = mcolors.ListedColormap(colors)
        # Use a logarithmic norm, avoiding zero values
        #norm = mcolors.LogNorm(vmin=10**-3, vmax=10**3)

        # Plot using imshow with the custom colormap and logarithmic normalization
        #plt.figure(figsize = (5,5))
        #plt.imshow(masked_data, cmap=custom_cmap, norm=norm, interpolation='nearest')
        #cbar = plt.colorbar()
        #cbar.set_label("P_T")
        #plt.xlabel("Eta")
        #plt.ylabel("Phi")
        #plt.gca().invert_yaxis()
        #plt.show()
        
        train_sr_1 = np.concatenate((signal_train_sr_1, bkg_train_sr_1))
        train_sr_2 = np.concatenate((signal_train_sr_2, bkg_train_sr_2))

        train_label_sr = np.ones(np.shape(train_sr_1)[0])
        
        train_sb_1 = np.concatenate((signal_train_sb_1, bkg_train_sb_1))
        train_sb_2 = np.concatenate((signal_train_sb_2, bkg_train_sb_2))

        train_label_sb = np.zeros(np.shape(train_sb_1)[0])
        
        #val_sr_1 = np.concatenate((signal_val_sr_1, bkg_val_sr_1))
        #val_sr_2 = np.concatenate((signal_val_sr_2, bkg_val_sr_2))

        #val_label_sr = np.ones(np.shape(val_sr_1)[0])
        
        #val_sb_1 = np.concatenate((signal_val_sb_1, bkg_val_sb_1))
        #val_sb_2 = np.concatenate((signal_val_sb_2, bkg_val_sb_2))

        #val_label_sb = np.zeros(np.shape(val_sb_1)[0])
        
        test_label_signal = np.ones(np.shape(signal_test_1)[0])
        test_label_bkg = np.zeros(np.shape(bkg_test_1)[0])
        
        x_train_1 = np.concatenate((train_sr_1, train_sb_1))
        x_train_2 = np.concatenate((train_sr_2, train_sb_2))
        y_train = np.concatenate((train_label_sr, train_label_sb))

        #x_val_1 = np.concatenate((val_sr_1, val_sb_1))
        #x_val_2 = np.concatenate((val_sr_2, val_sb_2))
        #y_val = np.concatenate((val_label_sr, val_label_sb))
        
        x_test_1 = np.concatenate((signal_test_1, bkg_test_1))
        x_test_2 = np.concatenate((signal_test_2, bkg_test_2))
        y_test = np.concatenate((test_label_signal, test_label_bkg))
        
        x_train_1 = x_train_1.reshape((np.shape(x_train_1)[0], p_1, p_1, 1))
        x_train_2 = x_train_2.reshape((np.shape(x_train_2)[0], p_1, p_1, 1))
        #x_val_1 = x_val_1.reshape((np.shape(x_val_1)[0], p_1, p_1, 1))
        #x_val_2 = x_val_2.reshape((np.shape(x_val_2)[0], p_1, p_1, 1))
        x_test_1 = x_test_1.reshape((np.shape(x_test_1)[0],p_1,p_1,1))
        x_test_2 = x_test_2.reshape((np.shape(x_test_2)[0],p_1,p_1,1))
        epsilon_results = []
        for i in range(10):
            cwola = CWoLA(p_1)
            loss_object = keras.losses.BinaryCrossentropy()
            optimizer = keras.optimizers.Adam(learning_rate = 1e-4)
            cwola.compile(loss = loss_object, optimizer = optimizer, metrics = ['accuracy'])
            early_stopping = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience=30, restore_best_weights=True)
            cwola.fit({'jet_1': x_train_1, 'jet_2': x_train_2}, y_train, validation_split = 0.2,
              shuffle = True, batch_size = 500, callbacks = [early_stopping], epochs = 1000)  
            #validation_data=({'jet_1': x_val_1, 'jet_2': x_val_2}, y_val), 
        
            x_predict = cwola.predict([x_test_1, x_test_2])
            fpr, tpr, th = roc_curve(y_test, x_predict)
            f = interpolate.interp1d(fpr, tpr)
            epsilon_s = f([0.001, 0.01, 0.1])
            print(epsilon_s)
            epsilon_results.append(epsilon_s)
        epsilon_results = np.array(epsilon_results)
        mean_results.append(np.mean(epsilon_results, axis = 0))
        std_results.append(np.std(epsilon_results, axis = 0))
    

mean_results = np.array(mean_results)
std_results = np.array(std_results)
os.chdir(r'/home/manhducnmd/pp_dijet/Results_full')
with open('cwola_mean_id10.npy','wb') as f:
    np.save(f, mean_results)
with open('cwola_std_id10.npy', 'wb') as g:
    np.save(g, std_results)
    

2024-07-03 14:42:17.695162: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1


Train on 40762 samples, validate on 10191 samples
Epoch 1/1000
40762/40762 [==============================] - 3s 70us/sample - loss: 0.6817 - accuracy: 0.6058 - val_loss: 1.0229 - val_accuracy: 0.0000e+00
Epoch 2/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6733 - accuracy: 0.6134 - val_loss: 0.9828 - val_accuracy: 0.0022
Epoch 3/1000
40762/40762 [==============================] - 1s 32us/sample - loss: 0.6700 - accuracy: 0.6138 - val_loss: 1.0631 - val_accuracy: 6.8688e-04
Epoch 4/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6706 - accuracy: 0.6129 - val_loss: 0.8977 - val_accuracy: 0.0084
Epoch 5/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6678 - accuracy: 0.6142 - val_loss: 0.8958 - val_accuracy: 0.0019
Epoch 6/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6690 - accuracy: 0.6129 - val_loss: 0.9626 - val_accuracy: 5.8875e-04
Epoch 7/1000
40762/40762 [======

Epoch 11/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6664 - accuracy: 0.6137 - val_loss: 0.8612 - val_accuracy: 0.0059
Epoch 12/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6658 - accuracy: 0.6142 - val_loss: 0.8893 - val_accuracy: 0.0024
Epoch 13/1000
40762/40762 [==============================] - 1s 32us/sample - loss: 0.6646 - accuracy: 0.6149 - val_loss: 0.8734 - val_accuracy: 0.0090
Epoch 14/1000
40762/40762 [==============================] - 1s 31us/sample - loss: 0.6670 - accuracy: 0.6136 - val_loss: 1.0448 - val_accuracy: 0.0013
Epoch 15/1000
40762/40762 [==============================] - 1s 32us/sample - loss: 0.6663 - accuracy: 0.6139 - val_loss: 0.8888 - val_accuracy: 0.0582
Epoch 16/1000
40762/40762 [==============================] - 1s 31us/sample - loss: 0.6651 - accuracy: 0.6142 - val_loss: 0.9705 - val_accuracy: 0.0025
Epoch 17/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.66

40762/40762 [==============================] - 1s 30us/sample - loss: 0.6585 - accuracy: 0.6202 - val_loss: 1.0702 - val_accuracy: 0.0044
Epoch 24/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6561 - accuracy: 0.6230 - val_loss: 1.0041 - val_accuracy: 0.0166
Epoch 25/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6575 - accuracy: 0.6222 - val_loss: 1.0299 - val_accuracy: 0.0057
Epoch 26/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6533 - accuracy: 0.6271 - val_loss: 0.8857 - val_accuracy: 0.0265
Epoch 27/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6512 - accuracy: 0.6280 - val_loss: 1.0206 - val_accuracy: 0.0067
Epoch 28/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6475 - accuracy: 0.6311 - val_loss: 0.8518 - val_accuracy: 0.1162
Epoch 29/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6431 - accuracy:

40762/40762 [==============================] - 1s 30us/sample - loss: 0.6180 - accuracy: 0.6533 - val_loss: 1.0146 - val_accuracy: 0.1203
Epoch 34/1000
40762/40762 [==============================] - 1s 30us/sample - loss: 0.6114 - accuracy: 0.6568 - val_loss: 1.0356 - val_accuracy: 0.1165
[0.0029  0.0329  0.22805]
Train on 40762 samples, validate on 10191 samples
Epoch 1/1000
40762/40762 [==============================] - 2s 46us/sample - loss: 0.6905 - accuracy: 0.6053 - val_loss: 0.9924 - val_accuracy: 2.9438e-04
Epoch 2/1000
40762/40762 [==============================] - 1s 31us/sample - loss: 0.6712 - accuracy: 0.6146 - val_loss: 0.9516 - val_accuracy: 0.0020
Epoch 3/1000
40762/40762 [==============================] - 1s 31us/sample - loss: 0.6701 - accuracy: 0.6143 - val_loss: 1.0267 - val_accuracy: 4.9063e-04
Epoch 4/1000
40762/40762 [==============================] - 1s 31us/sample - loss: 0.6700 - accuracy: 0.6130 - val_loss: 0.9357 - val_accuracy: 0.0034
Epoch 5/1000
40762/407

...

Epoch 5/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6683 - accuracy: 0.6155 - val_loss: 1.0711 - val_accuracy: 8.7745e-04
Epoch 6/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6689 - accuracy: 0.6161 - val_loss: 0.8912 - val_accuracy: 0.0012
Epoch 7/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6679 - accuracy: 0.6164 - val_loss: 0.8114 - val_accuracy: 0.0249
Epoch 8/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6664 - accuracy: 0.6167 - val_loss: 1.0049 - val_accuracy: 9.7494e-05
Epoch 9/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6666 - accuracy: 0.6155 - val_loss: 0.8897 - val_accuracy: 5.8497e-04
Epoch 10/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6659 - accuracy: 0.6165 - val_loss: 1.0059 - val_accuracy: 9.7494e-05
Epoch 11/1000
41025/41025 [==============================] - 1s 30us/sample -

Epoch 59/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.5266 - accuracy: 0.7072 - val_loss: 2.0317 - val_accuracy: 0.0830
Epoch 60/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.5236 - accuracy: 0.7090 - val_loss: 1.9397 - val_accuracy: 0.0708
Epoch 61/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.5140 - accuracy: 0.7131 - val_loss: 1.4780 - val_accuracy: 0.1647
Epoch 62/1000
41025/41025 [==============================] - 1s 31us/sample - loss: 0.5174 - accuracy: 0.7109 - val_loss: 2.0240 - val_accuracy: 0.0824
[0.00035 0.0212  0.1698 ]
Train on 41025 samples, validate on 10257 samples
Epoch 1/1000
41025/41025 [==============================] - 2s 45us/sample - loss: 0.6887 - accuracy: 0.6034 - val_loss: 1.0160 - val_accuracy: 6.8246e-04
Epoch 2/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6722 - accuracy: 0.6145 - val_loss: 1.2561 - val_accuracy: 0.0000e+00
Epoch 

41025/41025 [==============================] - 1s 30us/sample - loss: 0.6649 - accuracy: 0.6170 - val_loss: 0.9077 - val_accuracy: 9.7494e-04
Epoch 13/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6650 - accuracy: 0.6170 - val_loss: 0.9114 - val_accuracy: 0.0030
Epoch 14/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6647 - accuracy: 0.6167 - val_loss: 0.9737 - val_accuracy: 0.0011
Epoch 15/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6640 - accuracy: 0.6167 - val_loss: 0.8118 - val_accuracy: 0.1517
Epoch 16/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6674 - accuracy: 0.6141 - val_loss: 0.9239 - val_accuracy: 0.0019
Epoch 17/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6659 - accuracy: 0.6161 - val_loss: 0.8971 - val_accuracy: 0.0017
Epoch 18/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6646 - accur

41025/41025 [==============================] - 1s 30us/sample - loss: 0.6623 - accuracy: 0.6171 - val_loss: 1.0520 - val_accuracy: 0.0018
Epoch 21/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6617 - accuracy: 0.6176 - val_loss: 0.8967 - val_accuracy: 0.0104
Epoch 22/1000
41025/41025 [==============================] - 1s 31us/sample - loss: 0.6606 - accuracy: 0.6180 - val_loss: 0.9975 - val_accuracy: 0.0016
Epoch 23/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6595 - accuracy: 0.6192 - val_loss: 0.9566 - val_accuracy: 0.0120
Epoch 24/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6592 - accuracy: 0.6202 - val_loss: 0.9178 - val_accuracy: 0.0186
Epoch 25/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6599 - accuracy: 0.6203 - val_loss: 0.9128 - val_accuracy: 0.0328
Epoch 26/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6590 - accuracy:

41025/41025 [==============================] - 1s 30us/sample - loss: 0.6327 - accuracy: 0.6435 - val_loss: 0.9267 - val_accuracy: 0.1096
Epoch 37/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6274 - accuracy: 0.6458 - val_loss: 0.9722 - val_accuracy: 0.0536
[0.002   0.0315  0.18855]
Train on 41025 samples, validate on 10257 samples
Epoch 1/1000
41025/41025 [==============================] - 2s 45us/sample - loss: 0.6891 - accuracy: 0.6003 - val_loss: 0.8672 - val_accuracy: 9.7494e-05
Epoch 2/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6712 - accuracy: 0.6151 - val_loss: 1.3066 - val_accuracy: 0.0000e+00
Epoch 3/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6720 - accuracy: 0.6143 - val_loss: 0.9594 - val_accuracy: 0.0000e+00
Epoch 4/1000
41025/41025 [==============================] - 1s 30us/sample - loss: 0.6686 - accuracy: 0.6159 - val_loss: 1.1003 - val_accuracy: 0.0000e+00
Epoch 5/1000
4